# FHI-Aims Calculator

This example illustrates the usage of Workflow for efficiently performing density-functioncal theory (DFT) calculations with the all-electron electronic structure code [FHI-Aims](https://fhi-aims.org/).

At first, we initialize a `ConfigSet` with a couple of (periodic and non-periodic) systems stored in `Input_Structures.xyz`.
We also define an `OutputSpec` to handle the output that will additionally comprise values calculated by FHI-Aims.\
Next, we define various parameter that will be used during the calculations.
Most of them define parameter settings for the DFT calculation (that will be written in the `control.in` input file of FHI-Aims).
Note that for non-periodic systems the calculations will be performed with the exact same settings, but without parameter specific to periodic systems (e.g. `k_grid`, `compute_analytical_stress`, etc.).
The last parameter `calculator_exec` defines the command used to call FHI-Aims and start an individual calculation, including specifications for the parallelization of that specific calcuation and excluding any redirection of the output (i.e. without e.g. `>> aims.out`).\
Now we define a `calculator` of the form ` (calc_constructor, args, kwargs)` using the `Aims` calculator of Workflow and the parameter dictionary we have just defined.
Finally, we apply the `generic.calculate()` function to process all the input structures in parallel (defined via the `autopara_info` settings). The values of the calculated `properties` will be written into `Output_Structures.xyz` with keys that have the specified `output_prefix`. Note that for non-periodic systems the property "stress" will be ignored.

In [ ]:
import pathlib

import wfl
from wfl.calculators.aims import Aims
from wfl.calculators import generic
from wfl.autoparallelize import AutoparaInfo
from wfl.configset import ConfigSet, OutputSpec


workdir = pathlib.Path(wfl.__file__).parents[1]/"docs/source/examples_files/fhiaims_calculator"
inputs = ConfigSet(workdir/'./Input_Structures.xyz')
outputs = OutputSpec('Output_Structures.xyz')

aims_kwargs = {
        'xc':                        'pbe',
        'spin':                      'none',
        'relativistic':              'none',
        'charge':                    0.,
        'sc_iter_limit':             500,
        'occupation_type':           'gaussian 0.01',
        'charge_mix_param':          0.6,
        'mixer':                     'pulay',
        'n_max_pulay':               10,
        'sc_accuracy_rho':           1e-2,
        'sc_accuracy_eev':           1e-2,
        'sc_accuracy_etot':          1e-4,
        'sc_accuracy_forces':        1e-2,
        'sc_accuracy_stress':        1e-1,
        'compute_forces':            True,
        'compute_analytical_stress': True,
        'KS_method':                 'parallel',
        'k_grid_density':            1e-1,
        'calculator_exec':           'srun -n2 --exclusive --mem=2GB aims.210313.scalapack.mpi.x',
        }


calculator = (Aims, [], aims_kwargs)


generic.calculate(
    inputs = inputs,
    outputs = outputs,
    calculator = calculator,
    properties = ["energy", "forces", "stress"],
    output_prefix = "FHIaims_",
    autopara_info = AutoparaInfo(
        num_python_subprocesses = 4,
        num_inputs_per_python_subprocess=1
    )
)

In [ ]:
import os
import ase.io


assert os.path.exists(outputs.files[0]), f'{outputs.files[0]} not found!'
outputs_atoms = ase.io.read(outputs.files[0], ':')
assert len(list(outputs_atoms)) == len(list(inputs))
for atoms_i in outputs_atoms:
    assert not atoms_i.info.get('DFT_FAILED_AIMS', False), f'Calculation with ID_mol={atoms_i.info["ID_mol"]} failed.'
